In [1]:
import pandas as pd
import nltk 
import pickle 

In [2]:
personalitycsv = pd.read_csv("mbti_1.csv")

In [3]:
import re
types = personalitycsv['type']
uncleanedposts = personalitycsv['posts']
cleanedposts = []
counter = 0 
newtype = []
newposts = []
for s in uncleanedposts: 
    cleaned = s.split("|||")
    created = []
    for x in cleaned: 
        text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', x)
        if len(text.strip()) < 3: 
              continue 
        else: 
              created.append(text.strip())
    for x in created: 
        newtype.append(types[counter])
        newposts.append(x)
    counter += 1

In [4]:
cleaneddf = pd.DataFrame() 
cleaneddf['Types'] = newtype
cleaneddf = cleaneddf.assign(id=(cleaneddf['Types']).astype('category').cat.codes)
cleaneddf['Posts']=newposts
cleaneddf

,Types,id,Posts
0,INFJ,8,enfp and intj moments sportscenter not top ...
1,INFJ,8,What has been the most life-changing experienc...
2,INFJ,8,On repeat for most of today.
3,INFJ,8,May the PerC Experience immerse you.
4,INFJ,8,The last thing my INFJ friend posted on his fa...
...,...,...,...
410455,INFP,9,I was going to close my facebook a few months ...
410456,INFP,9,30 Seconds to Mars - All of my collections. It...
410457,INFP,9,"I have seen it, and i agree. I did actually th..."
410458,INFP,9,Ok so i have just watched Underworld 4 (Awaken...


In [5]:
#Training Sentiment Model To Assess Posts 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyze = SentimentIntensityAnalyzer()
pos = []
neg = []
neu = []
compound = []
for x in cleaneddf['Posts']:
    analyzed = analyze.polarity_scores(x)
    pos.append(analyzed['pos'])
    neg.append(analyzed['neg'])
    neu.append(analyzed['neu'])
    compound.append(analyzed['compound'])

cleaneddf['Positive']=pos
cleaneddf['Negative']=neg
cleaneddf['Neutral']=neu
cleaneddf['Compound']=compound

In [ ]:
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.840B.300d.txt'
word2vec_output_file = 'glove.840B.300d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

In [ ]:
#If File not located for pickle run this
from gensim.models import KeyedVectors
filename = 'glove.840B.300d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)
pickle.dump(model, open( "model.p", "wb" ) )

In [6]:
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
model = pickle.load( open( "model.p", "rb" ) )

In [8]:
import numpy as np 
charcounts = []
wordcounts = []
numbercounts = []
allmodels = []
for x in cleaneddf['Posts']: 
    spliter = x.split()
    newmodel = np.zeros(300)
    first = True
    for y in spliter:
        y = re.sub(r'[^\w\s]','',y)
        y = y.lower()
        try:
            if first:
                newmodel = np.array(model[str(y)])
                first = False
            else: 
                newmodel += np.array(model[str(y)]) 
        except:
            continue
    allmodels.append(newmodel/len(spliter))
    numbercounts.append(len([x for x in x.split() if x.isdigit()]))
    wordcounts.append(len(spliter))
    charcounts.append(len(x))
cleaneddf['Vectors'] = allmodels
cleaneddf['CharCounts'] = charcounts
cleaneddf['NumberCounts'] = numbercounts
cleaneddf['WordCounts'] = wordcounts

In [9]:
allvectors = []
for i in range(300):
    empty = []
    allvectors.append(empty)
for vector in cleaneddf['Vectors']:
    for i in range(300): 
        allvectors[i].append(vector[i])    

for i in range(300):
    name = "Vector" + str(i)
    cleaneddf[name] = allvectors[i]    

In [10]:
cleaneddf = cleaneddf.drop(['Vectors'],axis = 1)
cleaneddf

,Types,id,Posts,CharCounts,NumberCounts,WordCounts,Vector0,Vector1,Vector2,Vector3,...,Vector290,Vector291,Vector292,Vector293,Vector294,Vector295,Vector296,Vector297,Vector298,Vector299
0,INFJ,8,enfp and intj moments sportscenter not top ...,65,0,10,0.048666,0.101238,-0.126730,-0.103111,...,0.056108,-0.164767,-0.003561,0.040664,0.101166,0.147577,-0.051816,0.038400,0.082892,-0.078360
1,INFJ,8,What has been the most life-changing experienc...,61,0,10,0.069644,0.287454,-0.003381,-0.055506,...,-0.206936,0.000180,-0.106586,-0.132355,0.153329,0.067408,-0.021258,0.051582,0.005985,-0.030550
2,INFJ,8,On repeat for most of today.,28,0,6,-0.109645,0.309563,-0.250297,-0.100388,...,-0.205930,-0.171407,0.019444,-0.185498,0.014593,-0.031991,-0.028441,-0.097105,0.016200,0.116196
3,INFJ,8,May the PerC Experience immerse you.,36,0,6,0.262368,0.025354,-0.132030,-0.066374,...,-0.216144,0.045832,-0.170137,-0.002519,0.065560,-0.067423,-0.240428,0.070806,0.127976,-0.004138
4,INFJ,8,The last thing my INFJ friend posted on his fa...,107,0,19,0.070736,0.184186,-0.126542,0.127377,...,-0.099172,0.030649,0.107528,-0.086214,0.078186,-0.070215,0.032399,0.093196,-0.047283,0.185068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410455,INFP,9,I was going to close my facebook a few months ...,193,0,40,0.080480,0.134324,-0.269117,-0.010553,...,-0.219012,0.073492,0.065388,0.019547,0.137126,-0.073177,0.013745,-0.090762,0.050335,0.162630
410456,INFP,9,30 Seconds to Mars - All of my collections. It...,85,1,18,0.065014,0.159443,-0.114777,-0.006862,...,-0.152720,0.046057,0.063996,0.007415,0.063590,-0.099732,-0.054618,-0.003745,-0.060029,0.083143
410457,INFP,9,"I have seen it, and i agree. I did actually th...",199,0,38,0.009448,0.166724,-0.247294,-0.151581,...,-0.200789,0.064960,-0.039382,-0.041747,0.148716,-0.048417,-0.080719,-0.013402,0.124531,0.174318
410458,INFP,9,Ok so i have just watched Underworld 4 (Awaken...,200,1,41,-0.013228,0.129549,-0.177374,-0.113464,...,-0.235097,0.051023,-0.022789,-0.098781,0.156418,0.001369,-0.060064,-0.072731,0.069270,0.100613


In [11]:
x = cleaneddf.drop(['Types','id','Posts'],axis = 1)
y = cleaneddf['id']
labels = cleaneddf['Types']

In [12]:
#Figures Out Relevant Features and Removes Irrelevant Ones
import statsmodels.api as sm
cols = list(x.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = x[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

/Applications/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


['NumberCounts', 'WordCounts', 'Vector3', 'Vector9', 'Vector10', 'Vector12', 'Vector15', 'Vector16', 'Vector20', 'Vector23', 'Vector27', 'Vector28', 'Vector30', 'Vector31', 'Vector32', 'Vector36', 'Vector37', 'Vector38', 'Vector40', 'Vector46', 'Vector49', 'Vector50', 'Vector52', 'Vector55', 'Vector58', 'Vector65', 'Vector69', 'Vector70', 'Vector73', 'Vector74', 'Vector79', 'Vector83', 'Vector86', 'Vector87', 'Vector88', 'Vector89', 'Vector91', 'Vector96', 'Vector97', 'Vector98', 'Vector102', 'Vector103', 'Vector105', 'Vector113', 'Vector116', 'Vector120', 'Vector123', 'Vector124', 'Vector125', 'Vector128', 'Vector131', 'Vector138', 'Vector142', 'Vector143', 'Vector145', 'Vector146', 'Vector148', 'Vector149', 'Vector150', 'Vector152', 'Vector154', 'Vector155', 'Vector159', 'Vector160', 'Vector162', 'Vector164', 'Vector166', 'Vector169', 'Vector170', 'Vector171', 'Vector172', 'Vector173', 'Vector174', 'Vector175', 'Vector176', 'Vector177', 'Vector180', 'Vector181', 'Vector182', 'Vector1

In [13]:
x = cleaneddf[selected_features_BE]
y = cleaneddf['id']
labels = cleaneddf['Types']

In [14]:
x

,NumberCounts,WordCounts,Vector3,Vector9,Vector10,Vector12,Vector15,Vector16,Vector20,Vector23,...,Vector275,Vector277,Vector279,Vector282,Vector286,Vector287,Vector288,Vector289,Vector291,Vector295
0,0,10,-0.103111,1.103382,0.120112,-0.017139,-0.053014,0.195294,-0.009882,-0.192968,...,0.006881,-0.103204,0.149549,-0.301717,0.195879,0.011702,0.008102,0.077249,-0.164767,0.147577
1,0,10,-0.055506,2.347190,-0.194696,0.062586,-0.121125,-0.174713,-0.011405,-0.096722,...,0.003520,0.162255,0.119355,-0.118109,0.153282,-0.071058,0.194290,-0.095628,0.000180,0.067408
2,0,6,-0.100388,2.271400,-0.224235,0.091262,0.064457,-0.058925,-0.121899,0.003687,...,-0.057559,0.138972,0.282743,-0.005755,0.230963,-0.122787,-0.083324,0.121420,-0.171407,-0.031991
3,0,6,-0.066374,1.716712,-0.032925,0.019138,-0.185553,-0.181153,0.076267,-0.081551,...,-0.025825,0.083266,0.056332,0.110999,0.275457,-0.085238,0.284057,0.047923,0.045832,-0.067423
4,0,19,0.127377,2.337100,-0.270470,0.018476,0.028919,-0.124201,0.004940,0.043133,...,-0.026908,0.037661,0.171206,-0.114109,0.113601,-0.016611,0.021922,-0.009326,0.030649,-0.070215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410455,0,40,-0.010553,2.418188,-0.252297,0.140620,-0.040858,-0.090906,-0.013945,0.020258,...,0.001444,0.021013,0.214899,-0.026045,0.154671,-0.034403,0.090904,-0.047065,0.073492,-0.073177
410456,1,18,-0.006862,1.920768,-0.151364,0.085522,-0.121177,-0.123201,-0.068132,-0.058792,...,0.030366,0.003169,0.170454,-0.062679,0.099972,-0.196024,0.108989,-0.027115,0.046057,-0.099732
410457,0,38,-0.151581,2.414100,-0.194914,0.197825,-0.052595,-0.118497,0.161900,-0.067171,...,0.018227,0.000765,0.295033,-0.016460,0.235071,0.024222,0.122903,-0.065783,0.064960,-0.048417
410458,1,41,-0.113464,2.278018,-0.167932,0.156542,-0.091218,-0.026859,-0.008995,-0.030206,...,0.086333,0.000032,0.352357,-0.053209,0.145064,-0.110137,0.121512,-0.013895,0.051023,0.001369


In [15]:
y

0         8
1         8
2         8
3         8
4         8
         ..
410455    9
410456    9
410457    9
410458    9
410459    9
Name: id, Length: 410460, dtype: int8

In [16]:
import sklearn 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
#Logistic Regression 
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print(logreg.score(X_train, y_train))
print(logreg.score(X_test, y_test))

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.22874498530104437
0.22682843638844224


In [19]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train,y_train)
print(gnb.score(X_train, y_train))
print(gnb.score(X_test, y_test))

0.0687163994867547
0.06725137650440968


In [20]:
from sklearn.tree import DecisionTreeClassifier
dct = DecisionTreeClassifier()
dct.fit(X_train,y_train)
print(dct.score(X_train, y_train))
print(dct.score(X_test, y_test))

0.9848755055303805
0.14315645860741608


In [21]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
print(rfc.score(X_train, y_train))
print(rfc.score(X_test, y_test))

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9771930679400348
0.1735613701700531


In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
ada.fit(X_train,y_train)
print(ada.score(X_train, y_train))
print(ada.score(X_test, y_test))

0.22128018970585847
0.2197534473517517


In [23]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

0.21155126768341212
0.2114700579837256


In [24]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
lr.fit(X_train,y_train)
print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

0.010970255009872809
0.010731439378745322


In [25]:
from sklearn.naive_bayes import BernoulliNB
cb = BernoulliNB()
cb.fit(X_train, y_train)
print(cb.score(X_train, y_train))
print(cb.score(X_test, y_test))

0.21175266773863471
0.21141158699995127


In [26]:
from sklearn import linear_model
reg = linear_model.RidgeCV()
reg.fit(X_train, y_train)
print(reg.score(X_train, y_train))
print(reg.score(X_test, y_test))

0.010936248158451956
0.010827417926285055


In [27]:
from sklearn import linear_model
reg = linear_model.Lasso()
reg.fit(X_train, y_train)
print(reg.score(X_train, y_train))
print(reg.score(X_test, y_test))

0.0
-3.652401007192552e-06
